##Firebase

In [ ]:
from firebase import firebase
import csv
import os.path as pth
#realtime firebase to HS

firebase = firebase.FirebaseApplication("https://rvcapstone-default-rtdb.asia-southeast1.firebasedatabase.app", None)

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

timestamp = firebase.get('/Data/Timing','')
accx = firebase.get('/Data/Acceleration_X','')
accy = firebase.get('/Data/Acceleration_Y','')
accz = firebase.get('/Data/Acceleration_Z','')
rotx = firebase.get('/Data/Rotation_Roll','')
roty = firebase.get('/Data/Rotation_Pitch','')
rotz = firebase.get('/Data/Rotation_Yaw','')

timestamp_list = []
accx_list = []
accy_list = []
accz_list = []
rotx_list = []
roty_list = []
rotz_list = []

for value in timestamp.values():
    timestamp_list.append(value)
for value in accx.values():
    accx_list.append(value)
for value in accy.values():
    accy_list.append(value)
for value in accz.values():
    accz_list.append(value)
for value in rotx.values():
    rotx_list.append(value)
for value in roty.values():
    roty_list.append(value)
for value in rotz.values():
    rotz_list.append(value)


filename = "session"
filenum = 1
while pth.exists(pth.abspath(filename+str(filenum)+".csv")):
    filenum+=1
with open(filename+str(filenum)+'.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(timestamp_list) #row 1 - row 7 accordingly
    writer.writerow(accx_list)
    writer.writerow(accy_list)
    writer.writerow(accz_list)
    writer.writerow(rotx_list)
    writer.writerow(accy_list)
    writer.writerow(accz_list)
    
sessionname = filename + str(filenum)
print(sessionname)
#HS to firestore
#remember to give the mainserviceaccountfile to HS too with the code

if not firebase_admin._apps:
    cred = credentials.Certificate('/content/drive/MyDrive/Colab Notebooks/mainserviceaccount.json') 
    firebase_admin.initialize_app(cred)

db = firestore.client()


session2


In [ ]:
#run these with appropriate functions after running "Final Function"
session = {
'approach jump count':450,
'block jump count':150,
'average approach jump height': 33.5, 
'average block jump height': 30.0,
'max approach jump height': 42.0, 
'max approach block jump height': 30.5,
'total average jump height': 32.0,
'peak jump height': 37.0,
'total jumps': 600
}

db.collection('users').document('Bt1OuEGrCKm413CALIqG').collection('trainings').document('week4').collection('sessions').document(str(sessionname)).set(session)

#'Below Average',
#'A Lot More Work Needs To Be Done On Jumping Mechanics',
#'Average',
#'Focus On Improving Jumping Mechanics',
#'Above Average',
#'So Close to Personal Best',
#'Excellent',
#'It Is A New Personal Best',

#'Under Training',
#'Increase Jump Count For Training Session',
#'Positive Adaptation',
#'Jump Count Is Just Right With Low Chance Of Injury',
#'Over-Reaching',
#'Training Plan Needs To Be Altered As Athlete Has Moderate Chance of Injury',
#'Over-Training',
#'Training Plan Must Be Restructured As Athlete Has High Chance of Injury',

#'Low Readiness',
#'Weekly Jump Count Is Far From Competition Readiness'
#'Moderate Readiness',
#'Increase Weekly Jump Count'
#'Competition Ready',
#'Athlete Is Ready For Competition'
#'Excellent Readiness',
#'Athlete Is In Excellent Condition'

#mainuser = {
#'competition jump': 1800,
#'weeks to competition readiness': 3,
#'average jump height': 40,
#'peak jump height':48,
#'performance enhancement rating': 'Above Average',
#'injury management rating': 'Positive Adaptation',
#'competition readiness rating': 'Excellent Readiness',
#}

#feedback = {
#'id': 'week8',
#'total approach jumps':1677,
#'total block jumps':523,
#'total jumps': 2200,
#'peak jump height':46,
#'average jump height':0,
#'weeks to competition readiness': 0,
#'performance enhancement rating':'Above Average',
#'performance enhancement feedback': 'So Close to Personal Best',
#'injury management rating': 'Positive Adaptation',
#'injury management feedback': 'Jump Count Is Just Right With Low Chance Of Injury',
#'competition readiness rating': 'Excellent Readiness',
#'competition readiness feedback': 'Athlete Is In Excellent Condition'
#}

#db.collection('users').document('9qK74YxlBco6S3vEf2DG').add()
#db.collection('users').document('9qK74YxlBco6S3vEf2DG').collection('trainings').document('week1').set()
#db.collection('users').document('9qK74YxlBco6S3vEf2DG').collection('trainings').document('week8').collection('sessions').document('session5').set(session1)

#db.collection('users').document('test').collection('trainings').document(str(sessionname)).set(session)


update_time {
  seconds: 1626070791
  nanos: 591495000
}

##All Functions

In [ ]:
def data_processing_NEW(link):
  #time is already in ms
  df = pd.read_csv(link, header=None)
  df_t = df.T

  ls=[]
  #get time stamps
  timezero = df_t[0][0]
  for i in range(len(df_t)):
    ls.append(df_t[0][i]-timezero)
  df_t['Time(s)'] = ls


  df_t.loc[:, 'Time(s)'] /= 1000
  #remove irrelavant columns
  df_t.pop(0)

  # shift column 'Time(s)' to first position
  first_column = df_t.pop('Time(s)')
  df_t.insert(0, 'Time(s)', first_column)

  #rename columns
  df_t.columns = ['Time(s)', 'ax(g)', 'ay(g)', 'az(g)', 'rotX', 'rotY', 'rotZ']

  #create new dataframe without repeated timings
  #take average of repeated readings

  #new_df = pd.DataFrame(data, columns = ['Time(s)', 'ax(g)', 'ay(g)', 'az(g)'])
  new_df = pd.DataFrame()
  to_append=[]
  total = len(df_t)-1
  i=0

  #remove rotational data
  df_t.pop('rotX')
  df_t.pop('rotY')
  df_t.pop('rotZ')

  while i<total:
  #for i in range(len(df)-1):
    if df_t.iloc[i]['Time(s)']<df_t.iloc[i+1]['Time(s)'] and df_t.iloc[i]['Time(s)']>=0:
      new_df = new_df.append(df_t.iloc[i], ignore_index=True)
      i +=1
    else:
      #recalculate
      #append into new dataframe
      new_x = (df_t.iloc[i]['ax(g)'] + df_t.iloc[i+1]['ax(g)'])/2
      new_y = (df_t.iloc[i]['ay(g)'] + df_t.iloc[i+1]['ay(g)'])/2
      new_z = (df_t.iloc[i]['az(g)'] + df_t.iloc[i+1]['az(g)'])/2
      time = df_t.iloc[i]['Time(s)']
      #to_append.append(df.iloc[i]['Time(s)'], new_x, new_y, new_z)
      to_append.extend((time, new_x, new_y, new_z))
      a_series = pd.Series(to_append, index = df.columns)
      #print(a_series)
      new_df = new_df.append(a_series, ignore_index=True)
      #empty list
      to_append=[]
      i+=2

  #convert to Gs
  new_df.loc[:, 'ax(g)'] /= 9.81
  new_df.loc[:, 'ay(g)'] /= 9.81
  new_df.loc[:, 'az(g)'] /= 9.81
  return new_df

In [ ]:
def get_jump(new_df):
  import plotly.graph_objects as go
  import pandas as pd
  from scipy.signal import find_peaks
  #basic code to identify peaks
  #need more qualifications in case it identifies more jumps than expected (could combine with angular velocity for example)
  #jump acc >2g for taekoff in general (B jump)
  #find minima
  y2 = new_df['ax(g)']
  #find index of local minima based on conditions
  indices = find_peaks(y2, height=2, distance=15)
  return indices

In [ ]:
def find_takeoff(new_df, elem):
  #find takeoff
  pk_index = new_df.loc[new_df['ax(g)'] == elem].index[0] #retrieves first instance
  dat = new_df.iloc[pk_index-6: pk_index+1,]
  dat = dat.reset_index(drop=True)

  #look at 4 points before peak to find take-off velocity
  #loop to find points before and after 0.94
  x = []
  y = []
  for i in range(len(dat)-1):
    if dat.loc[i,"ax(g)"]<0.94 and dat.loc[i+1,"ax(g)"]>=0.94:
      x.append(dat.loc[i,"Time(s)"])
      x.append(dat.loc[i+1,"Time(s)"])
      y.append(dat.loc[i,"ax(g)"])
      y.append(dat.loc[i+1,"ax(g)"])
      break
    else:
      pass

  #interpolate acc at take-off
  x_takeoff = np.interp(0.94, x, y)
  return x_takeoff

In [ ]:
def find_landing(new_df, elem):
  #find landing
  pk_index = new_df.loc[new_df['ax(g)'] == elem].index[0] #retrieves first instance
  dat = new_df.iloc[pk_index: pk_index+7,]
  dat = dat.reset_index(drop=True)

  #look at 4 points before peak to find take-off velocity
  #loop to find points before and after 0.94
  x = []
  y = []
  for i in range(len(dat)-1):
    if dat.loc[i,"ax(g)"]>=0.94 and dat.loc[i+1,"ax(g)"]<0.94:
      x.append(dat.loc[i,"Time(s)"])
      x.append(dat.loc[i+1,"Time(s)"])
      y.append(dat.loc[i,"ax(g)"])
      y.append(dat.loc[i+1,"ax(g)"])
      break
    else:
      pass

  #interpolate acc at take-off
  x_landing = np.interp(0.94, x, y)
  return x_landing

In [ ]:
def jump_height(new_df, acc_ls):
  #ls contains a list of peak acc identified when counting jumps
  dist_ls=[]
  for elem in acc_ls:
    t_takeoff = find_takeoff(new_df, elem)
    t_landing = find_landing(new_df, elem)
    flight_time = t_landing-t_takeoff
    jump_height = (0.982*flight_time)/8
    
    #store each estimation in dist_ls
    dist_ls.append(jump_height)
  return dist_ls

In [ ]:
#normalize data
def normalize(df):
    result = df.copy()
    for feature_name in df.columns: #normalize by rows
      if feature_name != "Time": #only normalize accelerations
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


In [ ]:
#function for differentiating jumps

def ML_processing(new_df, all_acc_ls):
  lst=[]
  for elem in all_acc_ls:
    pk_index = new_df.loc[new_df['ax(g)'] == elem].index[0] #retrieves first instance
    shaped_data = new_df.iloc[pk_index-5: pk_index+20,]
    #we need to only look at x_axis
    df = shaped_data.iloc[:,3]
    df = pd.DataFrame(df)

    #transform dataframe into list to store multiple
    inp_norm = np.array(normalize(df))
    arr = inp_norm.reshape(1,25)
    lst.append(arr)
  
  return lst

In [ ]:
def diff_jumps(lst, filename):
  lst_class=[]
  x_test = np.vstack(lst)

  # load the model from disk
  loaded_model = pickle.load(open(filename, 'rb'))
  #put into model for classification
  #rfc_predict = loaded_model.predict(x)

  for i in rfc_predict:
    if i == 1:
      lst_class.append('Approach jump')
    else:
      lst_class.append('Block jump')

  #count different types of jumps
  #from collections import Counter
  #z = ['Block jump', 'Approach jump']
  #dict_class = Counter(z)
  #dict_class = {"Block jump":5, "Approach jump":1}
  #lst_class = ['Block jump', 'Approach jump', 'Block jump', 'Block jump', 'Block jump', 'Block jump']
  return dict_class,lst_class

In [ ]:
def get_jump_stats(jump_dict, class_lst, height):
  count_block = jump_dict.get("Block jump")
  count_approach = jump_dict.get("Approach jump")

  #split into block and approach jumps
  b_lst=[]
  a_lst=[]

  for i in range(len(class_lst)):
    if class_lst[i]=="Block jump":
      b_lst.append(height[i])
    else:
      a_lst.append(height[i])

  #find avg and max block jump height
  if len(b_lst)==0:
    avg_block_height=0
    max_block_height=0
  else:
    avg_block_height = sum(b_lst)/len(b_lst)
    max_block_height = max(b_lst)

  #find avg and max approach jump height
  if len(a_lst)==0:
      avg_approach_height=0
      max_approach_height=0
  else:
    avg_approach_height = sum(a_lst)/len(a_lst)
    max_approach_height = max(a_lst)

  return({"Block":[count_block, avg_block_height, max_block_height], 
          "Approach":[count_approach, avg_approach_height, max_approach_height]})

##Final functions to get the required variables

In [ ]:
link="session1.csv"
data_processing_NEW(link)
new_df = data_processing_NEW(link)
new_df

,Time(s),ax(g),ay(g),az(g)
0,0.000,3.190430,3.819491,3.337512
1,0.306,3.178966,3.903642,3.358245
2,0.583,3.235799,3.849005,3.230432
3,0.883,3.790953,3.707533,3.807052
4,1.179,2.645765,3.912668,2.302818
...,...,...,...,...
143,45.936,3.221408,3.864859,3.279947
144,46.242,3.225798,3.834126,3.242628
145,46.527,3.271654,3.811930,3.309217
146,46.797,3.231896,3.824857,3.314339


In [ ]:
#calling of basic functions
#new_df = data_processing(pd.read_excel('Book2.xlsx')) #should get from firebase
link="session1.csv"
get_csv()
data_processing_NEW(link)
new_df = data_processing_NEW(link)
jump_data = get_jump(new_df) 

#find jump height of all jumps
total_jump = len(jump_data[0])#total jumps
all_jump_acc = jump_data[1]["peak_heights"]#list of all recorded peak accelerations
height = jump_height(new_df, all_jump_acc)

#differentiate jumps
jump_lst = ML_processing(new_df, all_jump_acc)
filename = "/content/drive/MyDrive/finalized_model.sav"
jump_dict, class_lst = diff_jumps(jump_lst, filename)
final_dict = get_jump_stats(jump_dict, class_lst, height)
final_dict

#final_dict - dictionary of stats for different jumps
#[counts, avg height, max height]

{'Approach': [1, 0.21748649999999997, 0.21748649999999997],
 'Block': [5, 0.16062934999999998, 0.22335425]}